# Part 1: Train N-Gram Language Models and answer questions


This notebook has 20 points.

Here we examine how to build count-based MLE language models.


## Part 1.1: Language models

In [1]:
# load libraries
import nltk
from nltk.corpus import PlaintextCorpusReader

from nltk.util import ngrams
from nltk.lm.preprocessing import pad_both_ends

nltk.download('punkt') 

from tqdm import tqdm

# ngram:
_N = 3

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Download a wikipedia dataset:
#! wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
#! unzip wikitext-2-raw-v1.zip

## Preprocessing

In [3]:
# create a corpus reader
# this includes: sentence segmentation and word tokenization:
wikitext2 = PlaintextCorpusReader(
    'wikitext-2-raw',
    ['wiki.train.raw', 'wiki.valid.raw', 'wiki.test.raw'],
)
word_tokenizer = wikitext2._word_tokenizer

In [5]:
# training and test split:
train = wikitext2.sents('wiki.train.raw')
test = wikitext2.sents('wiki.test.raw')

# the vocabulary based on the training data:
vocab = nltk.lm.Vocabulary([
    word
    for sent in train
    for word in sent
], unk_cutoff=1)

In [9]:
# build n-grams
def build_ngrams(sent, n):
    # pad both ends for corner-ngrams:
    sent = ['<s>']*(n-1) + sent + ['</s>']*(n-1)
    # build the ngrams:
    return list(ngrams(sent, n))

In [10]:
# run this cell to inspect how it works:
sample = "Minecraft is a sandbox video game developed by Mojang."
sample_tokinized = word_tokenizer.tokenize(sample)
sample_trigrams = build_ngrams(sample_tokinized, n=3)
print('sample_tokinized:')
print(sample_tokinized)
print('sample_trigrams')
print(sample_trigrams)


sample_tokinized:
['Minecraft', 'is', 'a', 'sandbox', 'video', 'game', 'developed', 'by', 'Mojang', '.']
sample_trigrams
[('<s>', '<s>', 'Minecraft'), ('<s>', 'Minecraft', 'is'), ('Minecraft', 'is', 'a'), ('is', 'a', 'sandbox'), ('a', 'sandbox', 'video'), ('sandbox', 'video', 'game'), ('video', 'game', 'developed'), ('game', 'developed', 'by'), ('developed', 'by', 'Mojang'), ('by', 'Mojang', '.'), ('Mojang', '.', '</s>'), ('.', '</s>', '</s>')]


## Training the count model

In [11]:
%%time
# compare these two models:
models = {
    'plain': nltk.lm.MLE, # plain count-based ngrams
    'smoothing': nltk.lm.Laplace, # with laplace smoothing
    'smoothing+interpolation': nltk.lm.KneserNeyInterpolated, # Modified Kneser & Ney 
}

for lm_name in models:
    # build and train the language model:
    models[lm_name] = models[lm_name](_N, vocabulary=vocab)

    # train on all n-grams (equal or lower order): N, N-1, ..., 1.
    for n in tqdm(range(_N, 0, -1), desc=lm_name):
        models[lm_name].fit([build_ngrams(sent, n) for sent in train])


smoothing+interpolation: 100%|██████████| 3/3 [00:38<00:00, 12.93s/it]

CPU times: total: 1min 56s
Wall time: 1min 57s


#### Understand the models

In [12]:
# Understand how fit words:
# fit() method builds all kinds of count dictionaries:
(
    models['plain'].counts[1], # unigrams
    models['plain'].counts[2], # bi-grams for conditional count freq (w_{t} | w_{t-1})
    models['plain'].counts[3], # tri-grams for conditional count freq (w_{t} | w_{t-2} w_{t-1})
)

(FreqDist({'the': 113161, ',': 99925, '.': 78888, 'of': 56889, 'and': 50605, 'in': 39488, 'to': 39190, 'a': 34269, '=': 29570, '"': 28309, ...}),
 <ConditionalFreqDist with 75988 conditions>,
 <ConditionalFreqDist with 701600 conditions>)

In [13]:
# for example: 
# Count( word_3='numer'   | word_1 = 'A', word_2 ='large' ) = 4
# Count( word_3='variety' | word_1 = 'A', word_2 ='large' ) = 3
# ...
list(models['plain'].counts[3].items())[200]

(('A', 'large'),
 FreqDist({'number': 4, 'variety': 3, 'portion': 3, 'team': 1, 'tent': 1, 'oil': 1, 'pyramid': 1, 'camp': 1, 'rear': 1, 'network': 1, ...}))

In [14]:
# understand this:
models['plain'].counts[3][('A', 'large')]['number'] / sum(models['plain'].counts[3][('A', 'large')].values())

0.15384615384615385

In [15]:
models['plain'].score('number', ('A', 'large'))
# more details in chapter 3 equation 3.12.
# https://web.stanford.edu/~jurafsky/slp3/3.pdf

0.15384615384615385

In [16]:
# You can use the plain model for random language generation:
models['plain'].generate(10)

['began',
 'to',
 'spread',
 'their',
 'views',
 '.',
 '<UNK>',
 '<UNK>',
 'He',
 'refused']

## Testing

In [17]:
# Inspect log probabilities:
models['plain'].logscore('mind')

-14.527499220336294

In [18]:
sample = "Minecraft is a sandbox video game developed by Mojang."
sample_ngrams = [
    None,
    build_ngrams(word_tokenizer.tokenize(sample), n=1), # unigrams
    build_ngrams(word_tokenizer.tokenize(sample), n=2), # bigrams
    build_ngrams(word_tokenizer.tokenize(sample), n=3), # trigrams
]

In [19]:
for model_name in models:
    print(f"{model_name} model:")
    for n in range(1, _N+1):
        print(f"{n}-gram", models[model_name].perplexity(sample_ngrams[n]))
    print()

plain model:
1-gram inf
2-gram inf
3-gram inf

smoothing model:
1-gram 5089.599724571091
2-gram 4218.755058726706
3-gram 13170.588563844552

smoothing+interpolation model:


1-gram 1087.2475507780318
2-gram 334.25241384629607
3-gram 328.99011711235784



### Questions

1. Why these models have `<UNK>` token? What is the log-probability of `<UNK>` in three models? 


In [ ]:
The <UNK> token is used to represent unknown tokens in our code that means data that we have not encountered while we were training our model or are very rare words 
so we defined them as <unk>. It is necessary to have in order to avoid getting a lot of zeros as a result because that would lower the precision of our model.
So the <unk> token generalises unseen words and gives predictions. Generalization is good for a model as it learns to handle patterns instead of memorizing specific words.
Log-prob of <UNK> in plain model: -inf
Log-prob of <UNK> in LaPlace smoothing: -21.03873951979207
Log-prob of <UNK> in Interpolation and smoothing: -12.162899323232523 

In [20]:
for model_name in models:
    print(f"{model_name} model:",models[model_name].logscore('<UNK>'))
    print()

plain model: -inf

smoothing model: -21.03873951979207

smoothing+interpolation model: -12.162899323232523



#### You have already been introduced the concepts of model evaluation, training and test data, probabilities. Explore a related concept of ‘Perplexity’ and answer the following questions. 
#### See section 3.2.1 from Jurafsky & Martin Chapter 3 for some details on perplexity.
#### This might be helpful too. https://towardsdatascience.com/perplexity-intuition-and-derivation-105dd481c8f3


2. Why plain count-based MLE model fails to produce perplexities? What are the possible solutions for it? 


In [ ]:
One of the main reasons is the unseen n-grams since their probabilities will be 0 and the perplexity as a result will be -inf. Also, if the vocabulary is too big the model 
may not have all the necessary information so in the end the probabilities and perplexities it calculates are not trustworthy and if it is too small it cannot generalize.
A possible solutions is smoothing techniques as well interpolation and maybe neural language models that however have their own porblems.

3. Show with an example why Laplace smoothing can produce perplexity for unseen words? 

In [21]:
sample = "La playa al atardecer es un lugar tranquilo y hermoso."
sample_ngrams = [
    None,
    build_ngrams(word_tokenizer.tokenize(sample), n=1), # unigrams
    build_ngrams(word_tokenizer.tokenize(sample), n=2), # bigrams
    build_ngrams(word_tokenizer.tokenize(sample), n=3), # trigrams
]


In [22]:
for n in range(1, _N+1):
    print(f"{n}-gram", models["smoothing"].perplexity(sample_ngrams[n]))
    print()

1-gram 162025.13657586955

2-gram 29800.809435232753

3-gram 25384.451658389768



In [ ]:
As the example illustrates when using Laplace smoothing the perplexity is produced for unseen words. If we were to use the plain model then the 2-gram and 3-gram
perplexity would be -inf.

4. Why perplexity of bi-grams are lower than unigrams? 

In [ ]:
Perplexity of bigrams is lower because there is more context information whereas unigrams treat each word independently and smoothing makes probabilities bigger.
Also, bigrams reduce ambiguity and have a smaller vocabulary and that can lead to a higher probability so lower perplexity.


<ConditionalFreqDist with 75988 conditions>

## VG Part: neural language models and perplexity of conditional trigrams


The neural network below is based on Bengio et al. (2003). It is trained on moving windows described in chapter 9 figure 9.1 but with trigrams instead of 4-grams.
https://web.stanford.edu/~jurafsky/slp3/9.pdf

You don't need to train the model. However, a stand alone python code is provided in `bengio_lm.py` if you want to try training it on GPU.

Read the code below then report the perplexity of the language model on the sample sentence.

In [ ]:
import torch # neural network framework

# encoding the tokens:
vocab_list = [word for word, freq in vocab.counts.most_common() if freq > 1]
word2idx = {word: idx for idx, word in enumerate(['<s>', '</s>', vocab.unk_label]+vocab_list)}
idx2word = {idx: word for idx, word in enumerate(['<s>', '</s>', vocab.unk_label]+vocab_list)}

def token_encoder(tokens):
    if type(tokens) in {list, tuple}:
        return [word2idx[token] if token in word2idx else word2idx[vocab.unk_label] for token in tokens]
    elif type(tokens) == str:
        token = tokens
        return word2idx[token] if token in word2idx else word2idx[vocab.unk_label]
    print(type(tokens))

# moving window language model:
# https://jmlr.org/papers/volume3/tmp/bengio03a.pdf
class BengioLM(torch.nn.Module):
    def __init__(self, context_size=2, dim=50):
        super(BengioLM, self).__init__()
        # defining the parameters of the model
        self.C = torch.nn.Embedding(len(word2idx), dim) # C
        self.Hx_d = torch.nn.Linear(context_size*dim, dim) # d, H
        self.tanh = torch.nn.Tanh()
        self.Wx_Uf_b = torch.nn.Linear((context_size + 1) * dim, len(word2idx)) # b, U, W
        self.logsoftmax = torch.nn.LogSoftmax(dim=1)
        self.loss_fn = torch.nn.NLLLoss() # negative-log-likelihood loss
    
    def forward(self, context, target_idx=None):
        # function of the model
        batch_size = context.shape[0]
        x = self.C(context).view(batch_size,-1)
        x = torch.cat([x, self.tanh(self.Hx_d(x))], dim=-1)
        logprob = self.logsoftmax(self.Wx_Uf_b(x))
        
        if target_idx is None:
            return logprob
        else:
            loss = self.loss_fn(logprob, target_idx)
            return logprob, loss


ModuleNotFoundError: No module named 'torch'

#### The model is trained with Stochastic Gradient Descent with 10 epochs (skip this):

#### Load the model:

In [ ]:
# we ran the training code above on GPU and saved it in model.pt.
# load the pre-trained language model:
device = torch.device('cpu')
model = BengioLM()
model.load_state_dict(torch.load('model.pt', map_location=device))

NameError: name 'torch' is not defined

In [ ]:
# this is how you can get the conditional log-probabilities of all words in the sentence
# P(target | w0, w1):
for w0, w1, target in build_ngrams(word_tokenizer.tokenize(sample), n=3):
    logprobs = model.forward(torch.tensor([token_encoder([w0,w1])]))
    print(target, logprobs[0, token_encoder(target)])

Write a code here to report Perplexity of the sample sentence.

For more information got to chapter 3, section 3.2.1 and chapter 9, equation 9.12.

https://web.stanford.edu/~jurafsky/slp3/3.pdf

https://web.stanford.edu/~jurafsky/slp3/9.pdf

In [ ]:
# perplexity of a sentence 
# code here

Implement a generate function using pre-trained language model above

For hints see `bengio_lm.py`, for example how the training loop is implemented. The generation loop would be very similar to that.


In [ ]:
# code here